### Situation 1 : singel prompting

In [ ]:
%pip install --upgrade openai

In [3]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()   # take environment variables from .env.
API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=API_KEY)

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": """1. Extract all the characters, place names, time stamps, and key events from the provided text: “The $25 million settlement paid by President Donald Trump over the Trump University lawsuits continues to impact the 2028 US elections. Candidates are calling for stricter regulations on private universities, referencing the settlement approved by U.S. District Judge Gonzalo Curiel as a landmark victory for students.”.
2. Based on all the real information you know from the internet, assess the factualness of the extracted events. Show your analytic process.
3. Based on all the real information you know from the internet, assess the relationship between all characters, place names, time stamps, and key events. Show your analytic process.
4. Based on your analysis from steps 2 and 3, and all the information you currently have. Does this news article contain any misleading information or mismatched relationships? Show your analytic process and must respond with “YES” or “NO”.
5. How confident are you in your response, in terms of percentage? If your confidence is less than 50%, please change your answer to the opposite."""}],
    stream=False,
)

print(completion.choices[0].message.content)

1. 
Characters: President Donald Trump, U.S. District Judge Gonzalo Curiel
Place Names: Trump University
Time Stamps: 2028 US elections
Key Events: $25 million settlement paid by President Donald Trump over the Trump University lawsuits, candidates calling for stricter regulations on private universities referencing the settlement

2. To assess the factualness of the extracted events, we can check various news sources, official statements, court records, and credible sources to confirm the accuracy of the information provided in the text. In this case, we can verify the existence of the $25 million settlement paid by President Donald Trump over the Trump University lawsuits, the impact of this settlement on the 2028 US elections, and the reference to U.S. District Judge Gonzalo Curiel's approval of the settlement as a landmark victory for students.

3. Based on real information available, we can confirm that President Donald Trump was involved in a $25 million settlement over Trump Uni

### Situation 2 : recursive prompting

In [ ]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=API_KEY)

def chatgpt_response(prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            stream=False
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")
        return None

file_path = 'CoT_Prompt-Reuters-3.csv'
output_file_path = 'ChatGPT_Responses-test.csv'

df = pd.read_csv(file_path, encoding='utf-8-sig')
prompts = df['Reuters Version-Mix'].dropna().tolist()
output_df = pd.DataFrame(columns=df.columns.tolist() + ['ChatGPT Response'])

for i, prompt in enumerate(prompts):
    response = chatgpt_response(prompt)
    new_row = df.iloc[i].tolist() + [response]
    output_df.loc[i] = new_row
    output_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"Processed all prompts and saved responses to {output_file_path}")


Processed all prompts and saved responses to ChatGPT_Responses-test.csv
